In [1]:
import os

import torch
from datasets import load_dataset
from huggingface_hub import login
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from trl import SFTTrainer

In [2]:
# prevent env load failed
%load_ext dotenv
%dotenv

In [3]:
login(token=os.environ.get("HF_TOKEN", ""), add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/hermeschen/.cache/huggingface/token
Login successful


# Load Dataset

In [4]:
dataset = load_dataset("daily_dialog", split="train", num_proc=8, trust_remote_code=True)

In [5]:
dataset

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 11118
})

# Process Dataset

In [6]:
dataset = dataset.remove_columns("act")

## Process Emotion Feature (ID to Label)

In [7]:
dataset = dataset.rename_column("emotion", "emotion_id")

In [8]:
emotion_labels: list = dataset.features["emotion_id"].feature.names
emotion_labels[0] = "neutral"
emotion_labels

['neutral', 'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']

In [9]:
dataset = dataset.map(lambda samples: {
    "emotion": [[emotion_labels[emotion_id] for emotion_id in sample] for sample in samples]
}, input_columns="emotion_id", remove_columns="emotion_id", batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/11118 [00:00<?, ? examples/s]

In [10]:
dataset[0]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ',
  ' You know that is tempting but is really not good for our fitness . ',
  ' What do you mean ? It will help us to relax . ',
  " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
  " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
  ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
  " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
  ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
  " Good.Let ' s go now . ",
  ' All right . '],
 'emotion': ['neutral',
  'neutral',
  'neutral',
  'neutral',
  'neutral',
  'neutral',
  'happiness',
  'happiness',
  'happiness',
  'happiness']}

## Add Agent Feature

In [11]:
agents: dict = {0: 'user', 1: 'bot'}
dataset = dataset.map(lambda samples: {
    "agent": [[agents[i % 2] for i in range(len(sample))] for sample in samples["dialog"]]
}, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/11118 [00:00<?, ? examples/s]

In [12]:
dataset[0]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ',
  ' You know that is tempting but is really not good for our fitness . ',
  ' What do you mean ? It will help us to relax . ',
  " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
  " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
  ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
  " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
  ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
  " Good.Let ' s go now . ",
  ' All right . '],
 'emotion': ['neutral',
  'neutral',
  'neutral',
  'neutral',
  'neutral',
  'neutral',
  'happiness',
  'happiness',
  'happiness',
  'happiness'],
 'agent': ['user',
  'bot',
  'user',
  'bot',
  'u

## Remove Useless Spaces in Dialog

In [13]:
dataset = dataset.map(lambda samples: {
    "dialog": [[dialog.strip() for dialog in sample] for sample in samples]
}, input_columns="dialog", batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/11118 [00:00<?, ? examples/s]

In [14]:
dataset[0]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ?',
  'You know that is tempting but is really not good for our fitness .',
  'What do you mean ? It will help us to relax .',
  "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?",
  "I guess you are right.But what shall we do ? I don't feel like sitting at home .",
  'I suggest a walk over to the gym where we can play singsong and meet some of our friends .',
  "That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .",
  'Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .',
  "Good.Let ' s go now .",
  'All right .'],
 'emotion': ['neutral',
  'neutral',
  'neutral',
  'neutral',
  'neutral',
  'neutral',
  'happiness',
  'happiness',
  'happiness',
  'happiness'],
 'agent': ['user',
  'bot',
  'user',
  'bot',
  'user',
  'bot',
  'u

# Test Different Formats of Prompt

## Script Format
{agent}({emotion}): {dialog}

In [15]:
prompt_type = "script"

In [16]:
train_data = dataset.map(lambda samples: {
    "lines": [[f"{agent}({emotion}): {dialog}"
               for agent, emotion, dialog in zip(sample[0], sample[1], sample[2])]
              for sample in zip(samples["agent"], samples["emotion"], samples["dialog"])]
}, batched=True, num_proc=8)

In [17]:
enter = "\n"  # for Python 3.11
train_data = train_data.map(lambda samples: {
    "prompt": [f"""### {f'{enter}### '.join(sample)}""" for sample in samples]
}, input_columns="lines", remove_columns=["agent", "emotion", "dialog", "lines"], batched=True, num_proc=8)

In [18]:
print(train_data[0]["prompt"])

### user(neutral): Say , Jim , how about going for a few beers after dinner ?
### bot(neutral): You know that is tempting but is really not good for our fitness .
### user(neutral): What do you mean ? It will help us to relax .
### bot(neutral): Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?
### user(neutral): I guess you are right.But what shall we do ? I don't feel like sitting at home .
### bot(neutral): I suggest a walk over to the gym where we can play singsong and meet some of our friends .
### user(happiness): That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .
### bot(happiness): Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .
### user(happiness): Good.Let ' s go now .
### bot(happiness): All right .


## JSON format

In [19]:
prompt_type = "JSON"

In [20]:
train_data = dataset.map(lambda samples: {
    "lines": [[f'{{"agent": "{agent}", "emotion": "{emotion}", "dialog": "{dialog}"}}'
               for agent, emotion, dialog in zip(sample[0], sample[1], sample[2])]
              for sample in zip(samples["agent"], samples["emotion"], samples["dialog"])]
}, batched=True, num_proc=8)

In [21]:
enter = "\n"  # for Python 3.11
train_data = train_data.map(lambda samples: {
    "prompt": [f"""[{f',{enter}'.join(sample)}]""" for sample in samples]
}, input_columns="lines", remove_columns=["agent", "emotion", "dialog", "lines"], batched=True, num_proc=8)

In [22]:
print(train_data[0]["prompt"])

[{"agent": "user", "emotion": "neutral", "dialog": "Say , Jim , how about going for a few beers after dinner ?"},
{"agent": "bot", "emotion": "neutral", "dialog": "You know that is tempting but is really not good for our fitness ."},
{"agent": "user", "emotion": "neutral", "dialog": "What do you mean ? It will help us to relax ."},
{"agent": "bot", "emotion": "neutral", "dialog": "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?"},
{"agent": "user", "emotion": "neutral", "dialog": "I guess you are right.But what shall we do ? I don't feel like sitting at home ."},
{"agent": "bot", "emotion": "neutral", "dialog": "I suggest a walk over to the gym where we can play singsong and meet some of our friends ."},
{"agent": "user", "emotion": "happiness", "dialog": "That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them ."},
{"agent": "bot", "emotion": "happiness", "dialog": "Sounds grea

In [23]:
import json

print(json.dumps(eval(train_data[0]["prompt"]), indent=2))

[
  {
    "agent": "user",
    "emotion": "neutral",
    "dialog": "Say , Jim , how about going for a few beers after dinner ?"
  },
  {
    "agent": "bot",
    "emotion": "neutral",
    "dialog": "You know that is tempting but is really not good for our fitness ."
  },
  {
    "agent": "user",
    "emotion": "neutral",
    "dialog": "What do you mean ? It will help us to relax ."
  },
  {
    "agent": "bot",
    "emotion": "neutral",
    "dialog": "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?"
  },
  {
    "agent": "user",
    "emotion": "neutral",
    "dialog": "I guess you are right.But what shall we do ? I don't feel like sitting at home ."
  },
  {
    "agent": "bot",
    "emotion": "neutral",
    "dialog": "I suggest a walk over to the gym where we can play singsong and meet some of our friends ."
  },
  {
    "agent": "user",
    "emotion": "happiness",
    "dialog": "That's a good idea . I hear Mary and Sally often go there to

## History format

In [33]:
prompt_type = "history"

In [34]:
train_data = dataset.map(lambda samples: {
    "history": ["\n".join(sample[:-1]) for sample in samples]
}, input_columns="dialog", batched=True, num_proc=8)

In [35]:
enter = "\n"  # for Python 3.11
train_data = train_data.map(lambda samples: {
    "prompt": [f"HISTORY: {sample[0]}{enter}EMOTION: {sample[1][-1]}{enter}DIALOG: {sample[2][-1]}"
              for sample in zip(samples["history"], samples["emotion"], samples["dialog"])]
}, remove_columns=["agent", "emotion", "dialog", "history"], batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/11118 [00:00<?, ? examples/s]

In [36]:
print(train_data[0]["prompt"])

HISTORY: Say , Jim , how about going for a few beers after dinner ?
You know that is tempting but is really not good for our fitness .
What do you mean ? It will help us to relax .
Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?
I guess you are right.But what shall we do ? I don't feel like sitting at home .
I suggest a walk over to the gym where we can play singsong and meet some of our friends .
That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .
Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .
Good.Let ' s go now .
EMOTION: happiness
DIALOG: All right .


# Fine Tune

In [24]:
base_model_name: str = "meta-llama/Llama-2-7b-chat-hf"
fine_tuned_model_name: str = f"response_generator_llama_on_daily_dialog_type_{prompt_type}"

## Load Tokenizer

In [25]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, truncation=True, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16
tokenizer.truncation_side = "right"

## Configurations

In [26]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

In [27]:
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

In [28]:
num_train_epochs: int = 3

In [29]:
train_params = TrainingArguments(
    output_dir=f"./checkpoints_{fine_tuned_model_name}",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to=["tensorboard"],
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": True}
)

## Load Model

In [30]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quantization_config if torch.cuda.is_available() else None,
    device_map="auto" if torch.cuda.is_available() else "cpu",
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

## Setup Tuner

In [32]:
tuner = SFTTrainer(
    model=base_model,
    train_dataset=train_data,
    dataset_text_field="prompt",
    tokenizer=tokenizer,
    peft_config=peft_parameters,
    args=train_params,
    max_seq_length=1024
)

In [33]:
tuner.train()

Step,Training Loss
25,1.211500
50,0.911800
75,0.911400
100,0.843400
125,0.920900
150,0.843200
175,0.909000
200,0.856900
225,0.900200
250,0.831600


/home/hermeschen/.cache/pypoetry/virtualenvs/chat-bot-uhayQKRl-py3.11/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e445f5bf-db2f-4edb-a19e-051f583fee7a)') - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-chat-hf.
  warnings.warn(
/home/hermeschen/.cache/pypoetry/virtualenvs/chat-bot-uhayQKRl-py3.11/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=8340, training_loss=0.79868845985376, metrics={'train_runtime': 63106.1914, 'train_samples_per_second': 0.529, 'train_steps_per_second': 0.132, 'total_flos': 3.844355152962355e+17, 'train_loss': 0.79868845985376, 'epoch': 3.0})

In [34]:
tuner.model.save_pretrained(fine_tuned_model_name)